# Описание проекта

* **Сфера деятельности**: Интернет-магазин

* **Цель**: построить модель поиска токсичных комментариев в разделах с описаниями товаров в новом сервисе.

* **Ключевые критерии качества модели**
    - Метрика F1 $>=$ 0.75

* **Задачи**:
    - Загрузка данных
    - Подготовка данных к построению моделей
    - Обучение моделей
    - Выбор наилучшей модели и выводы
    

## Описание данных:
* **`toxic`** — целевой признак.

In [10]:
import numpy as np
import pandas as pd
import re
import nltk
import notebook
#  import pattern
import time
import spacy
from joblib import dump, load
from sklearn.pipeline import Pipeline

# from pattern.en import lemma



from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.utils import resample
from pymystem3 import Mystem
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import TfidfVectorizer 

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import f1_score

import notebook
from tqdm.notebook import tqdm
from tqdm import notebook 

## Загрузка данных

In [16]:
try:
    try:
        df = pd.read_csv('toxic_comments.csv')
    except:
        df = pd.read_csv('/datasets/taxi.csv')

    print('Вывод первых 5 строк из таблицы с данными.')
    display(df.head())
    print()
    print('Загрузка файла прошла успешно!!!')
except:
    print('При загрузке данных произошла ошибка. Проверьте наличие файла и/или путь к нему.')

Вывод первых 5 строк из таблицы с данными.


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0



Загрузка файла прошла успешно!!!


## Первичный анализ

Изучим данные по типам и наличию пропусков

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Проверим структуру целевого признака

In [18]:
print('Баланс классов')
print((df['toxic'].value_counts()/len(df)).map('{:.2%}'.format))

Баланс классов
0    89.83%
1    10.17%
Name: toxic, dtype: object


**Краткий вывод**:
* Пропуски в данных отсуствуют
* Типы данных соответствуют их содержимому
* В столбце **`text`** присутсвуют лишние для анализа символы
* Текстовые данные необходимо лемматизировать
* В данных присутсвут **сильный дисбаланс в классах** (соотношение близко 1:9). Это может негативно сказаться на прогнозной силе моделей обучения.

## Подготовка данных

In [19]:
# Функция очистки текстовой информации от сторонних символов
def clean(text):
    
    text = text.lower()    
    text = re.sub(r'(?:\n|\r)', ' ', text)
    text = re.sub(r"[^a-zA-Z ]+", "", text).strip()
        
    return text

In [20]:
df['clean_text'] = df['text'].apply(clean)
df['clean_text']

0         explanation why the edits made under my userna...
1         daww he matches this background colour im seem...
2         hey man im really not trying to edit war its j...
3         more i cant make any real suggestions on impro...
4         you sir are my hero any chance you remember wh...
                                ...                        
159566    and for the second time of asking when your vi...
159567    you should be ashamed of yourself   that is a ...
159568    spitzer   umm theres no actual article for pro...
159569    and it looks like it was actually you who put ...
159570    and  i really dont think you understand  i cam...
Name: clean_text, Length: 159571, dtype: object

In [21]:
# # Функция лемматизации
# def lemmatize(text):
#     word_list = text.split()
#     return " ".join([lemma(word) for word in word_list])

In [22]:
# import sys
# !{sys.executable} -m pip install spacy
# # Download spaCy's  'en' Model
# !{sys.executable} -m spacy download en

In [23]:
# Новая функция лемматизации
nlp = spacy.load('en_core_web_sm')

def lemmatize(text):
    temp = []
    for token in nlp(text):
        if token.is_stop == False:
            temp.append(token.lemma_)
    return " ".join(temp)

sentence = "The striped bats are hanging on their feet for best"
lemmatize(sentence)

'stripe bat hang foot good'

In [26]:
try:
    df['lemm_text'] = df['clean_text'].apply(lemmatize)
    display(df.head())
except Exception as error:
    print('Возникла ошибка!!!')
    print()
    print('ОШИБКА:', error)

,text,toxic,clean_text,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...,explanation edit username hardcore metallica f...
1,D'aww! He matches this background colour I'm s...,0,daww he matches this background colour im seem...,daww match background colour m seemingly stuck...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man im really not trying to edit war its j...,hey man m try edit war guy constantly remove r...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i cant make any real suggestions on impro...,not real suggestion improvement wonder secti...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...,sir hero chance remember page s


In [4]:
try:
    dump(df, '/datasets/toxic_comments_lemm.csv')
    df = load('/datasets/toxic_comments_lemm.csv')
except Exception as error:
    print('Возникла ошибка!!!')
    print()
    print('ОШИБКА:', error)
    print('Сохраню в свою дирректорию')
    dump(df, 'toxic_comments_lemm.csv')
    df = load('toxic_comments_lemm.csv')
    display(df.head())

Возникла ошибка!!!

ОШИБКА: [Errno 2] No such file or directory: '/datasets/toxic_comments_ppg.csv'
Сохраню в свою дирректорию


,text,toxic,clean_text,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...,explanation edit username hardcore metallica f...
1,D'aww! He matches this background colour I'm s...,0,daww he matches this background colour im seem...,daww match background colour m seemingly stuck...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man im really not trying to edit war its j...,hey man m try edit war guy constantly remove r...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i cant make any real suggestions on impro...,not real suggestion improvement wonder secti...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...,sir hero chance remember page s


<font color='purple'><b>ИСПРАВЛЕНИЕ ЛЕММАТИЗАЦИИ. Решил сохранить себе файл ,чтобы можно было его не потерять и начинать с этого момента.</b></font>

In [16]:
df = load('toxic_comments_lemm.csv')

In [17]:
# Функция балансировки классов путем увеличения выборки
rnd_st = 12345
def upsample(df):
    toxic_comments = df[df['toxic']==1]
    non_toxic = df[df['toxic']==0]
    toxic_comments_upsample = resample(toxic_comments,
                                       replace=True,
                                       n_samples=len(non_toxic),
                                       random_state=rnd_st)
    
    df_upsampled = pd.concat([non_toxic, toxic_comments_upsample])

    return df_upsampled
    

In [18]:
df_train, df_test = train_test_split(df, test_size = 0.25, random_state=rnd_st)

df_train_upsample = upsample(df_train)

print('Размеры сбалансированного набора данных обучающей выборки', df_train_upsample.shape)
print()
print('Баланс классов')
print((df_train_upsample['toxic'].value_counts()/len(df_train_upsample)).map('{:.2%}'.format))

print('Размеры сбалансированного набора данных тестовой выборки', df_test.shape)
print()
print('Баланс классов')
print((df_test['toxic'].value_counts()/len(df_test)).map('{:.2%}'.format))

x_train = df_train_upsample['lemm_text']
y_train = df_train_upsample['toxic']

x_test = df_test['lemm_text']
y_test = df_test['toxic']


Размеры сбалансированного набора данных обучающей выборки (215080, 4)

Баланс классов
0    50.00%
1    50.00%
Name: toxic, dtype: object
Размеры сбалансированного набора данных тестовой выборки (39893, 4)

Баланс классов
0    89.76%
1    10.24%
Name: toxic, dtype: object


## Обучение моделей

In [19]:
rnd_st=12345
cv = KFold(n_splits=4, shuffle=True, random_state=rnd_st)
stop_words = set(stopwords.words('english'))


estimators = {
    'log_reg': LogisticRegression(random_state=rnd_st),
    'dec_tree_class': DecisionTreeClassifier(random_state = rnd_st),
    'rand_forest_class': RandomForestClassifier(random_state = rnd_st),
    'lgmb_class': LGBMClassifier(boosting_type='gbdt', random_state = rnd_st),
    'cat_boost_class': CatBoostClassifier(silent=True, random_state = rnd_st)
}

params = {
    'log_reg': {
        'max_iter': [150], 
        'solver': ['liblinear']
    },
    'dec_tree_class': {
        'max_depth': [10, 20, 50]
    },
    'rand_forest_class': {
        'max_depth': [10, 20, 50],
        'n_estimators': [5, 10, 20]
        
    },
    'lgmb_class': {
        'n_estimators': [10, 20],
        'num_leaves': [10, 20],
    },
    'cat_boost_class': {
        'n_estimators': [10, 20, 50]        
    }
}

# vectorizers = {
#     'count_vect': CountVectorizer(stop_words=stop_words, dtype=np.float64),
#     'tf_idf': TfidfVectorizer(stop_words=stop_words, dtype=np.float64)
# }


# pipes_list = []
# pipes_params_list =[]



# pipe = Pipeline([
#     ('vectorizer', TfidfVectorizer(ngram_range=(1, 1))),
#     ('model', LogisticRegression(max_iter=150, solver='liblinear', random_state=42))
# ])

# params = {
#     'vectorizer__ngram_range': [(1, 1), (3, 1), (5, 2)],
#     'model': [LogisticRegression(max_iter=150, solver='liblinear', random_state=42)],
#     'model__C': [1, 5, 10, 25],
# }

In [20]:
# len(vectorizers.keys())
# for vectorizer in vectorizers.keys():
#     for estimator in estimators.keys()

### CountVectorizer

In [24]:
corpus_train = x_train.values
corpus_test = x_test.values

stop_words = set(stopwords.words('english'))
count_vect = CountVectorizer(stop_words=stop_words, dtype = np.float64)

count_vect_train = count_vect.fit_transform(corpus_train)
count_vect_test = count_vect.transform(corpus_test)

In [26]:
count_vect_best_models = []
for model in list(estimators.keys()):
    try:
        print('МОДЕЛЬ', model.upper())

        start = time.time()

        grid = GridSearchCV(estimator=estimators[model], param_grid=params[model], cv=cv, scoring='f1', n_jobs=-1)
        grid = grid.fit(count_vect_train, y_train)

        best_model = grid.best_estimator_
        predictions = best_model.predict(count_vect_train)

        print(f'Время перебора моделей: {(time.time()-start):0.2f}')
        print(f'Лучшие параметры модели: {model.upper()}: {grid.best_params_}')
        print(f'f1_score равен: {f1_score(y_train, predictions)}')
        print('_'*100)

        count_vect_best_models.append(grid.best_estimator_)
    
    except Exception as error:
        
        print('Возникла ошибка!!! В модели', model.upper())
        print()
        print('ОШИБКА:', error)
        print('НУЖНА ПОМОЩЬ!!!')
        print('_'*100)
       

МОДЕЛЬ LOG_REG


D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Время перебора моделей: 82.92
Лучшие параметры модели: LOG_REG: {'max_iter': 150, 'solver': 'liblinear'}
f1_score равен: 0.9636853538696775
____________________________________________________________________________________________________
МОДЕЛЬ DEC_TREE_CLASS
Время перебора моделей: 140.34
Лучшие параметры модели: DEC_TREE_CLASS: {'max_depth': 50}
f1_score равен: 0.8533401694019661
____________________________________________________________________________________________________
МОДЕЛЬ RAND_FOREST_CLASS
Время перебора моделей: 111.89
Лучшие параметры модели: RAND_FOREST_CLASS: {'max_depth': 50, 'n_estimators': 20}
f1_score равен: 0.8696054888507719
____________________________________________________________________________________________________
МОДЕЛЬ LGMB_CLASS
Время перебора моделей: 111.42
Лучшие параметры модели: LGMB_CLASS: {'n_estimators': 20, 'num_leaves': 20}
f1_score равен: 0.7805290280531789
_____________________________________________________________________________

In [27]:
print(count_vect_best_models)
print()
for model in count_vect_best_models:
    print(f'f1_score модели {model} равно {f1_score(model.predict(count_vect_test), y_test)}')
    print()

[LogisticRegression(max_iter=150, random_state=12345, solver='liblinear'), DecisionTreeClassifier(max_depth=50, random_state=12345), RandomForestClassifier(max_depth=50, n_estimators=20, random_state=12345), LGBMClassifier(n_estimators=20, num_leaves=20, random_state=12345), <catboost.core.CatBoostClassifier object at 0x000002B683D2A610>]

f1_score модели LogisticRegression(max_iter=150, random_state=12345, solver='liblinear') равно 0.751398179624959

f1_score модели DecisionTreeClassifier(max_depth=50, random_state=12345) равно 0.6662587198629298

f1_score модели RandomForestClassifier(max_depth=50, n_estimators=20, random_state=12345) равно 0.4187317321133562

f1_score модели LGBMClassifier(n_estimators=20, num_leaves=20, random_state=12345) равно 0.6819824470831183

f1_score модели <catboost.core.CatBoostClassifier object at 0x000002B683D2A610> равно 0.7225363041791376



**Краткий вывод**
* После перебора моделей на основе признаков, сформированных по принципу **"мешка слов"** (CountVectorizer), наилучший результат и на обучающей и на тестовой выборках по метрике **f1** показала модель **логистической регрессии**. 
    - f1_score на обучающей выборке: **0.964** >0.75
    - f1_score на тестовой выборке: **0.751** > 0.75


### TfidfVectorizer

In [28]:
corpus_train = x_train.values
corpus_test = x_test.values

stop_words = set(stopwords.words('english'))
tf_idf = TfidfVectorizer(stop_words=stop_words, dtype = np.float64)

tf_idf_train = tf_idf.fit_transform(corpus_train)
tf_idf_test = tf_idf.transform(corpus_test)

In [29]:
tf_idf_best_models = []
for model in list(estimators.keys()):
    try:
        print('МОДЕЛЬ', model.upper())

        start = time.time()

        grid = GridSearchCV(estimator=estimators[model], param_grid=params[model], cv=cv, scoring='f1', n_jobs=-1)
        grid = grid.fit(tf_idf_train, y_train)

        best_model = grid.best_estimator_
        predictions = best_model.predict(tf_idf_train)

        print(f'Время перебора моделей: {(time.time()-start):0.2f}')
        print(f'Лучшие параметры модели: {model.upper()}: {grid.best_params_}')
        print(f'f1_score равен: {f1_score(y_train, predictions)}')
        print('_'*100)

        tf_idf_best_models.append(grid.best_estimator_)
    
    except Exception as error:
        
        print('Возникла ошибка!!! В модели', model.upper())
        print()
        print('ОШИБКА:', error)
        print('НУЖНА ПОМОЩЬ!!!')
        print('_'*100)

МОДЕЛЬ LOG_REG
Время перебора моделей: 10.90
Лучшие параметры модели: LOG_REG: {'max_iter': 150, 'solver': 'liblinear'}
f1_score равен: 0.9735658172772376
____________________________________________________________________________________________________
МОДЕЛЬ DEC_TREE_CLASS
Время перебора моделей: 148.43
Лучшие параметры модели: DEC_TREE_CLASS: {'max_depth': 50}
f1_score равен: 0.8605699657811561
____________________________________________________________________________________________________
МОДЕЛЬ RAND_FOREST_CLASS
Время перебора моделей: 120.11
Лучшие параметры модели: RAND_FOREST_CLASS: {'max_depth': 50, 'n_estimators': 20}
f1_score равен: 0.8851164491086213
____________________________________________________________________________________________________
МОДЕЛЬ LGMB_CLASS
Время перебора моделей: 136.03
Лучшие параметры модели: LGMB_CLASS: {'n_estimators': 20, 'num_leaves': 20}
f1_score равен: 0.7898731692421781
______________________________________________________________

D:\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [      nan       nan 0.8895855]
  warnings.warn(


Время перебора моделей: 1213.25
Лучшие параметры модели: CAT_BOOST_CLASS: {'n_estimators': 50}
f1_score равен: 0.8936636075371953
____________________________________________________________________________________________________


In [30]:
print(tf_idf_best_models)
print()
for model in tf_idf_best_models:
    print(f'f1_score модели {model} равно {f1_score(model.predict(tf_idf_test), y_test)}')
    print()

[LogisticRegression(max_iter=150, random_state=12345, solver='liblinear'), DecisionTreeClassifier(max_depth=50, random_state=12345), RandomForestClassifier(max_depth=50, n_estimators=20, random_state=12345), LGBMClassifier(n_estimators=20, num_leaves=20, random_state=12345), <catboost.core.CatBoostClassifier object at 0x000002B682959F10>]

f1_score модели LogisticRegression(max_iter=150, random_state=12345, solver='liblinear') равно 0.7550818591363587

f1_score модели DecisionTreeClassifier(max_depth=50, random_state=12345) равно 0.6607341490545049

f1_score модели RandomForestClassifier(max_depth=50, n_estimators=20, random_state=12345) равно 0.4365691489361702

f1_score модели LGBMClassifier(n_estimators=20, num_leaves=20, random_state=12345) равно 0.6765036888833312

f1_score модели <catboost.core.CatBoostClassifier object at 0x000002B682959F10> равно 0.7212034698583507



**Краткий вывод**
* После перебора моделей на основе признаков, сформированных по принципу **"TF-IDF"** (TfidfVectorizer), наилучший результат и на обучающей и на тестовой выборках по метрике **f1** показала модель **логистической регрессии**. 
    - f1_score на обучающей выборке: **0.974** >0.75
    - f1_score на тестовой выборке: **0.755** > 0.75
* Удалось построить и перебрать модели всех классов.
    - Однако перебо моделей класса **CatBoostClassifier** осуществлялся весьма долго (**примерно 27 мин**)


## ОБЩИЙ ВЫВОД
* На основе полученных данных о содержании и токсичности комментариев были построены несколько классов моделей классификации комментариев по их "токсичности" **(логистическая регрессия, дерево решений, случайны лес, LGBMClassifier, CatBoostClassifier)**. В рамках перебора моделей использовались два принципа формирования прзнаков из текстовых данных **("мешок слов" и TF-IDF)**.
* Все модели оценивались с учетом значительного дисбаланса классов путем увеличения выборки (upsample метод).
* В ходе процесса лемматизации возникла ошибка.
    - ОШИБКА: "generator raised StopIteration"
    - Причина ошибки не утановлена
    - **Повторный запуск кода решил данную проблему** (поэтому лемматизация проведена в структуре `try ... except`
* Пришлось изменить способ лемматизации из-за ограничений JupyterHub. Время работы кода было увеличено.
* Не удалось построить модель **LGBMClassifier** на основе признаков, сформированных по принципу **"мешка слов"**:
    - ОШИБКА: "Expected np.float32 or np.float64, met type(int64)"
    - Причина ошибки была устранена (добавили параметр dtupe=np.float64 в функции векторизации)
* На основе полученных результатов оценивания моделии и тестирования **наилучшей оказалась модель логистической регрессии на основе признаков, свормированных по принципу TF-IDF**.
    - f1_score на обучающей выборке: **0.974** >0.75
    - f1_score на тестовой выборке: **0.755** > 0.75